In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns
sns.set(style="whitegrid", color_codes=True)
sns.set(font_scale=1)

train = pd.read_json(r"C:\JupyterNotebooks\Two Sigma\train.json")
test = pd.read_json(r"C:\JupyterNotebooks\Two Sigma\test.json")

In [2]:
print (f'There are {train.shape[0]} rows and {train.shape[1]} attributes.')
print (len(train['listing_id'].unique()))
train = train.set_index('listing_id')
print (f'There are {test.shape[0]} rows and {test.shape[1]} attributes.')

There are 49352 rows and 15 attributes.
49352
There are 74659 rows and 14 attributes.


In [3]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from textblob import TextBlob
from sklearn.metrics import accuracy_score
import time

train = pd.read_json(r"C:\JupyterNotebooks\Two Sigma\train.json")
test = pd.read_json(r"C:\JupyterNotebooks\Two Sigma\test.json")

print (f'There are {train.shape[0]} rows and {train.shape[1]} attributes.')
train = train.set_index('listing_id')
print (f'There are {test.shape[0]} rows and {test.shape[1]} attributes.')

def pre_processing(data):
       
    data['nb_images'] = data['photos'].apply(len)
    data['nb_features'] = data['features'].apply(len)
    data['nb_description'] = data['description'].apply(lambda x: len(x.split(' ')))
    data['description_len'] = data['description'].apply(len)
    
    def room_price(x, y):
        if y == 0:
            return 0
        return x/y
    
    def sentiment_analysis(x):
        if len(x) == 0:
            return 0
        return TextBlob(x[0]).sentiment.polarity
    
    data = data.join(data['description'].apply(
                         lambda x: TextBlob(x).sentiment.polarity).rename('sentiment'))
    data['price_room'] = data.apply(lambda row: 
                                    room_price(row['price'],row['bedrooms']), axis=1)
       
    return data[important_features]

def print_scores(test_name, train, test):
    print ('{0} train score: {1}\n{0} test score: {2}\n'.format(test_name,
                                                               train,
                                                               test))

def classification(train_data, test_data, target, test_size=0.2, random_state=19):    
    # Split data into X and y
    X = train_data
    y = target

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
                                                        random_state=random_state)
    
    # Support vector machine
    svm_model = svm.SVC(decision_function_shape='ovo', tol=0.00000001)
    svm_model = svm_model.fit(X_train, y_train)
    print_scores("Support Vector Machine",
                 svm_model.score(X_train, y_train),
                 accuracy_score(y_test, svm_model.predict(X_test)))

    # Random Forest
    random_forest = RandomForestClassifier(n_estimators=10)
    random_forest = random_forest.fit(X_train, y_train)
    print_scores("Random Forest",
                 random_forest.score(X_train, y_train),
                 accuracy_score(y_test, random_forest.predict(X_test)))

    # GradientBoostingClassifier
    gradientB_model = GradientBoostingClassifier(n_estimators=20,
                                      learning_rate=1.0,
                                      max_depth=1,
                                      random_state=0).fit(X_train, y_train)
    gradientB_model = gradientB_model.fit(X_train, y_train)
    print_scores("Gradient Boosting Classifier",
                 gradientB_model.score(X_train, y_train),
                 accuracy_score(y_test, gradientB_model.predict(X_test)))


There are 49352 rows and 15 attributes.
49352
There are 74659 rows and 14 attributes.


In [4]:
global important_features
important_features = ['bathrooms', 'bedrooms', 'price', 'price_room',
                      'latitude','longitude', 'nb_images','nb_features',
                      'sentiment', 'nb_description', 'description_len']

In [5]:
processed_train_data = pre_processing(train)

In [6]:
processed_train_data = processed_train_data[['bathrooms', 'bedrooms', 'price', 'price_room',
                            'latitude','longitude', 'nb_images','nb_features', 
                            'nb_description', 'description_len','sentiment']]

In [7]:
processed_test_data = pre_processing(test)

In [8]:
target = train['interest_level']  
classification(processed_train_data, processed_test_data, target)

Support Vector Machine train score: 0.969251032142043
Support Vector Machine test score: 0.7070205652922703

Random Forest train score: 0.9625896000607888
Random Forest test score: 0.7126937493668322

Gradient Boosting Classifier train score: 0.7040095235682987
Gradient Boosting Classifier test score: 0.704386586971938

